# Outlier Analysis

Questions:
* If you found NULL values after you joined the datasets, what did you do with them? How many were there?
After joining the datasets, there were 750332 null values in the SA2 dataset and 182 in the origninal dataset, these comprised of approximatley 2% of the total dataset. for null values in the SA2 dataset, the postcodes were used to determine which state the purchase came from. then averaging over all the data in that state the other columns were populated.

* Was there any missing data that shouldn’t be missing after joining to your external
dataset? If anything was missing, how much was there and what did you do about it?
there was only 182 rows of missing data once joined, due to the extremely low number and the fact that the whole purchase was null, they were removed.

* If you decided to omit outliers, what does the distribution look like prior and after?



In [7]:
from pyspark.sql import SparkSession, functions as F, DataFrame
from pyspark.sql.functions import col

spark = (
    SparkSession.builder.appName("Customer_Analysis")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

In [8]:
data = spark.read.parquet('../data/curated/final_data.parquet')

In [112]:
data.limit(5)

SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed Year 12,Did Not Attend School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P
null,852,1140,6421,23338656015,240.79240453805204,c22bbaa1-8e70-49b...,2021-08-19,Iaculis LLC,"[watch, clock, ...",Christopher Powell,880 Brandon Stree...,Undisclosed,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,7321,94215,11809,49891706470,48.86426156234827,5cfef23a-a07c-455...,2021-05-21,Non Vestibulum In...,[tent and awning ...,Amy Nguyen,85806 Melton Squares,Female,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,6872,799687,24034,38090089066,118.19708950112054,ece422e2-c9e8-42c...,2021-04-02,Interdum Feugiat ...,"[furniture, home...",Angela Little,337 Carter Walk,Female,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,7330,334995,4041,33357097824,24.599565257317405,7fd18add-d03c-479...,2021-07-29,Sem Ut Incorporated,"[hobby, toy and ...",Eric Cooper,137 Brian Turnpike,Male,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,852,1140,6421,48534649627,269.8021779483706,dcc21448-04c5-42d...,2021-08-14,Dignissim Maecena...,"[opticians, opti...",Christopher Powell,880 Brandon Stree...,Undisclosed,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [97]:
from pyspark.sql.functions import isnan, when, count, col
data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns])

SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed Year 12,Did Not Attend School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P
750332,182,182,182,182,182,182,182,182,182,182,182,182,750332,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333


## SA2_CODE_2021 / SA2_NAME_2021

In [99]:
from pyspark.sql.functions import col
data.filter(col("SA2_CODE_2021").isNull()).count()/data.count()

0.019819309386088536

In [9]:
import pyspark.sql.functions as f
import pandas as pd
data = data.withColumn(
    'state',
    F.when(((col('postcode') >= 1000) & (col('postcode') <= 1999)) | ((col('postcode') >= 2000) & (col('postcode') <= 2599)) | ((col('postcode') >= 2619) & (col('postcode') < 2899)) | ((col('postcode') >= 2921) & (col('postcode') < 2999)), 'NSW')\
    .when(((col('postcode') >= 200) & (col('postcode') <= 299)) | ((col('postcode') >= 2600) & (col('postcode') <= 2618)) | ((col('postcode') >= 2900) & (col('postcode') < 2920)), 'ACT')\
    .when(((col('postcode') >= 3000) & (col('postcode') <= 3999)) | ((col('postcode') >= 8000) & (col('postcode') <= 8999)), 'VIC')\
    .when(((col('postcode') >= 4000) & (col('postcode') <= 4999)) | ((col('postcode') >= 9000) & (col('postcode') <= 9999)), 'QLD')\
    .when(((col('postcode') >= 5000) & (col('postcode') <= 5999)), 'SA')\
    .when(((col('postcode') >= 6000) & (col('postcode') <= 6999)), 'WA')\
    .when(((col('postcode') >= 7000) & (col('postcode') <= 7999)), 'TAS')\
    .otherwise('NT')
)
data = data.withColumn(
    'SA2_NAME_2021',
    F.when( (col('state') == 'VIC') & (col('SA2_NAME_2021').isNull()), 'Victoria')\
    .when( (col('state') == 'NSW') & (col('SA2_NAME_2021').isNull()), 'New South Wales')\
    .when( (col('state') == 'QLD') & (col('SA2_NAME_2021').isNull()), 'Queensland')\
    .when( (col('state') == 'NT') & (col('SA2_NAME_2021').isNull()), 'Northern Territory')\
    .when( (col('state') == 'WA') & (col('SA2_NAME_2021').isNull()), 'Western Austraia')\
    .when( (col('state') == 'SA') & (col('SA2_NAME_2021').isNull()), 'South Australia')\
    .when( (col('state') == 'TAS') & (col('SA2_NAME_2021').isNull()), 'Tasmania')\
    .otherwise('Australian Capital Territory')
)
data.limit(5)

22/09/11 15:42:13 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed Year 12,Did Not Attend School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P,state
null,852,1140,6421,23338656015,240.79240453805204,c22bbaa1-8e70-49b...,2021-08-19,Iaculis LLC,"[watch, clock, ...",Christopher Powell,880 Brandon Stree...,Undisclosed,Northern Territory,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,NT
null,7321,94215,11809,49891706470,48.86426156234827,5cfef23a-a07c-455...,2021-05-21,Non Vestibulum In...,[tent and awning ...,Amy Nguyen,85806 Melton Squares,Female,Tasmania,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,TAS
null,6872,799687,24034,38090089066,118.19708950112054,ece422e2-c9e8-42c...,2021-04-02,Interdum Feugiat ...,"[furniture, home...",Angela Little,337 Carter Walk,Female,Western Austraia,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,WA
null,7330,334995,4041,33357097824,24.599565257317405,7fd18add-d03c-479...,2021-07-29,Sem Ut Incorporated,"[hobby, toy and ...",Eric Cooper,137 Brian Turnpike,Male,Tasmania,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,TAS
null,852,1140,6421,48534649627,269.8021779483706,dcc21448-04c5-42d...,2021-08-14,Dignissim Maecena...,"[opticians, opti...",Christopher Powell,880 Brandon Stree...,Undisclosed,Northern Territory,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,NT


In [10]:
data = data.withColumn("Median_tot_prsnl_inc_weekly", data.Median_tot_prsnl_inc_weekly.cast('float'))
data = data.withColumn("Median_rent_weekly", data.Median_rent_weekly.cast('float'))
data = data.withColumn("Median_mortgage_repay_monthly",data.Median_mortgage_repay_monthly.cast('float'))
data = data.withColumn("Median_age_persons",data.Median_age_persons.cast('float'))
data = data.withColumn("Median_tot_hhd_inc_weekly",data.Median_tot_hhd_inc_weekly.cast('float'))
data = data.withColumn("Average_household_size",data.Average_household_size.cast('float'))
data = data.withColumnRenamed("Completed Year 12", "Completed_Year_12")
data = data.withColumn("Completed_Year_12",data.Completed_Year_12.cast('float'))
data = data.withColumnRenamed("Did Not Attend School", "Did_Not_Attend_School")
data = data.withColumn("Did_Not_Attend_School",data.Did_Not_Attend_School.cast('float'))
data = data.withColumn("TOT_P_P",data.TOT_P_P.cast('float'))
data = data.withColumn("Age_0_4_yr_P",data.Age_0_4_yr_P.cast('float'))
data = data.withColumn("Age_5_14_yr_P",data.Age_5_14_yr_P.cast('float'))
data = data.withColumn("Age_15_19_yr_P",data.Age_15_19_yr_P.cast('float'))
data = data.withColumn("Age_20_24_yr_P",data.Age_20_24_yr_P.cast('float'))
data = data.withColumn("Age_25_34_yr_P",data.Age_25_34_yr_P.cast('float'))
data = data.withColumn("Age_35_44_yr_P",data.Age_35_44_yr_P.cast('float'))
data = data.withColumn("Age_45_54_yr_P",data.Age_45_54_yr_P.cast('float'))
data = data.withColumn("Age_55_64_yr_P",data.Age_55_64_yr_P.cast('float'))
data = data.withColumn("Age_65_74_yr_P",data.Age_65_74_yr_P.cast('float'))
data = data.withColumn("Age_75_84_yr_P",data.Age_75_84_yr_P.cast('float'))
data = data.withColumn("Age_75_84_yr_P",data.Age_75_84_yr_P.cast('float'))
data = data.withColumn("Age_75_84_yr_P",data.Age_75_84_yr_P.cast('float'))
data = data.withColumn("Age_85ov_P",data.Age_85ov_P.cast('float'))


In [147]:
data.groupBy('state').mean()

state,avg(consumer_id),avg(user_id),avg(merchant_abn),avg(dollar_value),avg(Median_rent_weekly),avg(Median_mortgage_repay_monthly),avg(Median_age_persons),avg(Median_tot_hhd_inc_weekly),avg(Average_household_size),avg(Completed_Year_12),avg(Did_Not_Attend_School),avg(TOT_P_P),avg(Age_0_4_yr_P),avg(Age_5_14_yr_P),avg(Age_15_19_yr_P),avg(Age_20_24_yr_P),avg(Age_25_34_yr_P),avg(Age_35_44_yr_P),avg(Age_45_54_yr_P),avg(Age_55_64_yr_P),avg(Age_65_74_yr_P),avg(Age_75_84_yr_P),avg(Age_85ov_P)
NT,700007.6518392774,12368.083358220294,5.645368617969288E10,149.51633014332364,87.9500782220102,478.8819373802993,31.990788744449045,1277.1489375135925,3.4840981139623093,709.3371263408235,37.410346351627226,3169.560259009562,189.11927628855855,366.8003339343216,203.83062654777856,274.28154311330616,669.4101639505272,502.12604617554,435.1914439853237,342.297306777604,125.70409630778082,42.06529257837986,17.79855903130985
ACT,757414.1779197967,11791.032408837731,5.580714729691582E10,163.03838550411336,402.75962734886497,1949.3786282130754,35.847500803127446,2284.063630317043,2.3944066075994628,2628.0384505748584,15.697829519797658,4344.574722525123,239.78317368818747,505.6231499789603,265.0335458416097,347.3054418105886,756.2062386035084,634.215360321432,540.4276438729294,456.12947319364196,354.1367714729131,180.13218346598134,65.95499771504586
SA,758814.4433247212,11813.756111437275,5.587278183620381E10,159.07315739562105,199.18416140802304,1069.03480178228,47.13529300669113,1226.6664763894582,2.258507024866534,2182.486092534747,28.611025993301595,6149.016459231163,308.97961133732844,716.0268275706521,314.06660450068756,293.23685147100554,678.3452006489837,683.5231756730539,779.9096740045158,935.3453428263774,835.5851795679263,440.1321804131087,164.24808606188628
TAS,716152.2497466155,11750.990522782971,5.575075575214775E10,158.1451704134173,247.91508634682842,1182.5427094138233,46.957218003637045,1225.7466943272127,2.277830912050027,1654.4454957915325,10.896212247181818,5081.844906685248,248.56150697188284,578.2532956432001,260.97548927230065,235.90282385669303,562.1103254601073,560.1099960673461,654.4874437485319,815.6952620757551,717.6227201824329,345.01931698510606,102.33022732851386
WA,760070.0500151335,12247.06972745315,5.592799003636488E10,160.249466482334,235.5527510544241,1437.028984063036,42.581815939994776,1530.0211433587413,2.4394982812246875,2478.71725636885,28.07392873978152,6548.73302066646,374.6621691528398,813.4824127179769,335.70097131695496,332.4277592342616,855.2077198837901,877.2738058527914,884.7022050086551,897.2414754612912,717.4611607539548,350.85974351244175,110.15619551797715
QLD,746628.021668531,12040.489371904825,5.582572139488198E10,159.0428476651104,244.15880563861472,1314.0752483250237,41.714496689157386,1466.238010910226,2.520670685421044,2720.037294460769,30.33145496559106,7765.427526962527,439.58056954924746,1024.785338407404,445.7255284506005,409.41527879509994,929.9685759334338,936.0585798833365,1033.8205188239615,1094.5369516697826,881.3959722137969,441.0361307271616,127.40040478879166
VIC,762374.7597994036,12229.776857401534,5.58478263309848E10,156.3518395435879,276.0413448062838,1452.774237045356,45.21639873221335,1471.0240264506567,2.392283172952558,3932.661585145785,57.57916776424643,9672.860644336348,513.2277276302308,1153.960691114338,527.4418103363422,499.6983871436459,1193.4942538066878,1170.5818006432676,1235.8209612267644,1337.3127722661827,1185.39246565005,624.0616807641218,231.72313373161808
NSW,750941.615912689,11867.755331151793,5.590629894817144E10,159.11939422487316,351.68120935335656,1814.3416669907058,44.26223298817287,1625.0275687971643,2.4792831829350526,4383.78123400978,52.53136657119299,10867.791191028951,589.9850943527954,1333.9051728556644,611.0508426045313,572.7601993856221,1324.9165633055868,1326.75730943634,1387.2278518599692,1482.5984677226215,1289.7086481298677,696.7499085326247,252.29505486275033


In [11]:
columns = ["Median_tot_prsnl_inc_weekly", "Median_rent_weekly", "Median_mortgage_repay_monthly", "Median_age_persons", "Median_tot_hhd_inc_weekly",\
           "Average_household_size", "Completed_Year_12", "Did_Not_Attend_School", "TOT_P_P", "Age_5_14_yr_P", "Age_15_19_yr_P", "Age_20_24_yr_P",\
           "Age_25_34_yr_P", "Age_35_44_yr_P", "Age_45_54_yr_P", "Age_55_64_yr_P", "Age_65_74_yr_P", "Age_75_84_yr_P", "Age_85ov_P"]

for column in columns:
    data = data.withColumn(
        column,
        F.when( (col('SA2_NAME_2021') == 'Northern Territory'), data.groupBy('state').mean(column).collect()[0][1])\
        .when( (col('SA2_NAME_2021') == 'Australian Capital Territory'), data.groupBy('state').mean(column).collect()[1][1])\
        .when( (col('SA2_NAME_2021') == 'South Australia'), data.groupBy('state').mean(column).collect()[2][1])\
        .when( (col('SA2_NAME_2021') == 'Tasmania'), data.groupBy('state').mean(column).collect()[3][1])\
        .when( (col('SA2_NAME_2021') == 'Western Austraia'), data.groupBy('state').mean(column).collect()[4][1])\
        .when( (col('SA2_NAME_2021') == 'Queensland'), data.groupBy('state').mean(column).collect()[5][1])\
        .when( (col('SA2_NAME_2021') == 'Victoria'), data.groupBy('state').mean(column).collect()[6][1])\
        .otherwise(data.groupBy('state').mean(column).collect()[7][1])
    )
data

SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed_Year_12,Did_Not_Attend_School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P,state
null,852,1140,6421,23338656015,240.79240453805204,c22bbaa1-8e70-49b...,2021-08-19,Iaculis LLC,"[watch, clock, ...",Christopher Powell,880 Brandon Stree...,Undisclosed,Northern Territory,645.9527090071066,87.9500782220102,478.8819373802993,31.990788744449045,1277.1489375135925,3.4840981139623093,709.3371263408235,37.410346351627226,3169.560259009562,null,366.8003339343216,203.83062654777856,274.28154311330616,669.4101639505272,502.12604617554,435.1914439853237,342.297306777604,125.70409630778082,42.06529257837986,17.79855903130985,NT
null,7321,94215,11809,49891706470,48.86426156234827,5cfef23a-a07c-455...,2021-05-21,Non Vestibulum In...,[tent and awning ...,Amy Nguyen,85806 Melton Squares,Female,Tasmania,654.7627923426741,247.91508634682842,1182.5427094138233,46.957218003637045,1225.7466943272127,2.277830912050027,1654.4454957915325,10.896212247181818,5081.844906685248,null,578.2532956432001,260.97548927230065,235.90282385669303,562.1103254601073,560.1099960673461,654.4874437485319,815.6952620757551,717.6227201824329,345.01931698510606,102.33022732851386,TAS
null,6872,799687,24034,38090089066,118.19708950112054,ece422e2-c9e8-42c...,2021-04-02,Interdum Feugiat ...,"[furniture, home...",Angela Little,337 Carter Walk,Female,Western Austraia,836.7966870108314,235.5527510544241,1437.028984063036,42.581815939994776,1530.0211433587413,2.4394982812246875,2478.71725636885,28.07392873978152,6548.73302066646,null,813.4824127179769,335.70097131695496,332.4277592342616,855.2077198837901,877.2738058527914,884.7022050086551,897.2414754612912,717.4611607539548,350.85974351244175,110.15619551797715,WA
null,7330,334995,4041,33357097824,24.599565257317405,7fd18add-d03c-479...,2021-07-29,Sem Ut Incorporated,"[hobby, toy and ...",Eric Cooper,137 Brian Turnpike,Male,Tasmania,654.7627923426741,247.91508634682842,1182.5427094138233,46.957218003637045,1225.7466943272127,2.277830912050027,1654.4454957915325,10.896212247181818,5081.844906685248,null,578.2532956432001,260.97548927230065,235.90282385669303,562.1103254601073,560.1099960673461,654.4874437485319,815.6952620757551,717.6227201824329,345.01931698510606,102.33022732851386,TAS
null,852,1140,6421,48534649627,269.8021779483706,dcc21448-04c5-42d...,2021-08-14,Dignissim Maecena...,"[opticians, opti...",Christopher Powell,880 Brandon Stree...,Undisclosed,Northern Territory,645.9527090071066,87.9500782220102,478.8819373802993,31.990788744449045,1277.1489375135925,3.4840981139623093,709.3371263408235,37.410346351627226,3169.560259009562,null,366.8003339343216,203.83062654777856,274.28154311330616,669.4101639505272,502.12604617554,435.1914439853237,342.297306777604,125.70409630778082,42.06529257837986,17.79855903130985,NT
null,7321,94215,11809,49891706470,48.86426156234827,5cfef23a-a07c-455...,2021-05-21,Non Vestibulum In...,[tent and awning ...,Amy Nguyen,85806 Melton Squares,Female,Tasmania,654.7627923426741,247.91508634682842,1182.5427094138233,46.957218003637045,1225.7466943272127,2.277830912050027,1654.4454957915325,10.896212247181818,5081.844906685248,null,578.2532956432001,260.97548927230065,235.90282385669303,562.1103254601073,560.1099960673461,654.4874437485319,815.6952620757551,717.6227201824329,345.01931698510606,102.33022732851386,TAS
null,6872,799687,24034,89726005175,58.42108971934746,912a54f6-e059-47f...,2021-03-27,Est Nunc Consulting,[tent and awning ...,Angela Little,337 Carter Walk,Female,Western Austraia,836.7966870108314,235.5527510544241,1437.028984063036,42.581815939994776,1530.021143358741

## Postcode
remove all istances in a null postcode as there is not way to determine the location of the purchase with the postcode.

In [12]:
data = data.na.drop(subset=["postcode"])

In [13]:
print(data.agg({'postcode': 'min'}))
print(data.agg({'postcode': 'max'}))

+-------------+
|min(postcode)|
+-------------+
|         1001|
+-------------+



+-------------+
|max(postcode)|
+-------------+
|         9999|
+-------------+



According to AusPost, all post between 1000-9999 are valid, thus we will keep all.
## ABN
All ABNs are 11 digits long, thus we will verify this:

In [14]:
print(data.agg({'merchant_abn': 'min'}))
print(data.agg({'merchant_abn': 'max'}))

+-----------------+
|min(merchant_abn)|
+-----------------+
|      10023283211|
+-----------------+



+-----------------+
|max(merchant_abn)|
+-----------------+
|      99990536339|
+-----------------+



## Dollar Value

In [48]:
print(data.agg({'dollar_value': 'min'}))
print(data.agg({'dollar_value': 'max'}))

+--------------------+
|   min(dollar_value)|
+--------------------+
|9.756658099412162E-8|
+--------------------+



+------------------+
| max(dollar_value)|
+------------------+
|105193.88578925544|
+------------------+



the above minmum dollar value

In [71]:
data.where(col("dollar_value") <= 0.05).limit(10)

SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed Year 12,Did Not Attend School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P
null,5005,35789,9327,85139489422,0.002359119240645712,e30b6907-d82e-456...,2021-07-23,Ut Institute,"[cable, satellit...",Brianna Peters,118 Wagner Loop S...,Female,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,6980,63528,11753,98269572896,0.04146230443737651,b770d9f6-6f81-443...,2021-04-10,Eget Laoreet Posu...,"[furniture, home...",Steven Lane,84534 Melinda For...,Male,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,6900,867075,3889,49536733338,0.03565006025930135,7c71001f-5bcc-43a...,2021-06-28,Duis A Ltd,"[stationery, off...",Mary Leonard MD,367 Michael Courts,Female,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,6944,976661,8878,40590483301,0.003797908334501...,212f7cfa-8b71-420...,2021-07-27,Donec Tempus Corp...,"[cable, satellit...",Robert Williams,1695 Kendra Harbors,Male,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,1173,172938,23735,45629217853,0.034259091137831287,082a0a8b-e662-483...,2021-05-27,Lacus Consulting,"[gift, card, no...",Lisa Lopez,05080 April Spring,Female,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,1335,1094971,13524,70038202360,0.008066279230355394,d8cc3f9f-78d8-454...,2021-07-04,Varius Orci Insti...,[tent and awning ...,John Myers,1658 Alfred Tunnel,Male,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,3402,337063,19598,17324645993,0.028403696461256386,2f0dfa31-9efd-4ec...,2021-06-18,Eget Metus In Cor...,[tent and awning ...,William Bailey,269 Trevino Vista,Male,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,1480,348279,22079,63290521567,0.019027178794988053,438f67ed-5158-4e9...,2021-03-18,Vehicula Pellente...,[artist supply an...,Caitlin Henry,6656 Craig Juncti...,Undisclosed,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,1700,1323531,17223,47086412084,0.03531825399883669,794a7e02-2b0f-496...,2021-03-19,Eros Limited,[digital goods: b...,Danielle Wilson,487 Victoria Plaz...,Female,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,1227,1370566,23927,29521780474,0.017035451448531396,68182197-5f63-4e1...,2021-05-23,At Sem Corp.,"[cable, satellit...",George King,2225 Carpenter Fo...,Male,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [ ]:
data.where(col("dollar_value") <= 0.05).count()

All these purchases seem unreasonable due the the limited things you can buy for 5c, thus they will be removed from the dataset.

In [15]:
data  = data[data['dollar_value'] >= 0.05]

In [73]:
data.where(col("dollar_value") >= 50000).limit(10)

SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed Year 12,Did Not Attend School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P
null,1212,1319595,23700,11590404675,65660.03308418578,1d8f2322-d8c8-46c...,2021-08-26,Arcu Sed PC,[antique shops - ...,Angela Harris,572 Shannon Plaza,Undisclosed,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,6913,1380769,18916,83199298021,59223.92657075801,61fe4fa3-f199-4a1...,2021-08-08,Ligula Elit Preti...,[antique shops - ...,Tracy Short,1636 Davis Plains,Female,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
604031097,7466,165107,11702,53918538787,77134.2146046811,8584d8d1-061b-41c...,2021-07-27,In Tempus Inc.,[antique shops - ...,Ana Robinson,77844 Garcia Vall...,Female,West Coast (Tas.),548,160,650,47,958,2,1030,16,4267,208,462,172,200,521,454,629,726,576,254,59
604031097,7466,165107,11702,53918538787,77134.2146046811,8584d8d1-061b-41c...,2021-07-27,In Tempus Inc.,[antique shops - ...,Ana Robinson,77844 Garcia Vall...,Female,West Coast (Tas.),548,160,650,47,958,2,1030,16,4267,208,462,172,200,521,454,629,726,576,254,59
401051019,5069,1277505,10428,54860127682,51618.337208028475,572fa5c4-72f8-4ad...,2021-07-17,Laoreet Posuere E...,[antique shops - ...,Richard Jones,5032 William Glen,Male,St Peters - Marden,954,320,2080,41,1879,2.3,8201,77,13515,595,1453,769,856,1942,1682,1725,1851,1393,829,416
401051019,5069,1277505,10428,54860127682,51618.337208028475,572fa5c4-72f8-4ad...,2021-07-17,Laoreet Posuere E...,[antique shops - ...,Richard Jones,5032 William Glen,Male,St Peters - Marden,954,320,2080,41,1879,2.3,8201,77,13515,595,1453,769,856,1942,1682,1725,1851,1393,829,416
401051019,5069,1277505,10428,54860127682,51618.337208028475,572fa5c4-72f8-4ad...,2021-07-17,Laoreet Posuere E...,[antique shops - ...,Richard Jones,5032 William Glen,Male,St Peters - Marden,954,320,2080,41,1879,2.3,8201,77,13515,595,1453,769,856,1942,1682,1725,1851,1393,829,416
401051019,5069,1277505,10428,54860127682,51618.337208028475,572fa5c4-72f8-4ad...,2021-07-17,Laoreet Posuere E...,[antique shops - ...,Richard Jones,5032 William Glen,Male,St Peters - Marden,954,320,2080,41,1879,2.3,8201,77,13515,595,1453,769,856,1942,1682,1725,1851,1393,829,416
401051019,5069,1277505,10428,54860127682,51618.337208028475,572fa5c4-72f8-4ad...,2021-07-17,Laoreet Posuere E...,[antique shops - ...,Richard Jones,5032 William Glen,Male,St Peters - Marden,954,320,2080,41,1879,2.3,8201,77,13515,595,1453,769,856,1942,1682,1725,1851,1393,829,416
401051019,5069,1277505,10428,54860127682,51618.337208028475,572fa5c4-72f8-4ad...,2021-07-17,Laoreet Posuere E...,[antique shops - ...,Richard Jones,5032 William Glen,Male,St Peters - Marden,954,320,2080,41,1879,2.3,8201,77,13515,595,1453,769,856,1942,1682,1725,1851,1393,829,416


Al instances above $50,000 come from anitque shops or jewelry shops and thus is plausible for the individuals to be spending large amounts of money. 

## Order Date 

In [74]:
print(data.agg({'order_datetime': 'min'}))
print(data.agg({'order_datetime': 'max'}))

+-------------------+
|min(order_datetime)|
+-------------------+
|         2021-02-28|
+-------------------+

+-------------------+
|max(order_datetime)|
+-------------------+
|         2021-08-27|
+-------------------+



Based on the above all values are in 2021.

## Gender

In [82]:
from pyspark.sql.functions import count

data.groupBy('gender').count()

gender,count
Undisclosed,3786857
Female,16928185
Male,17129766


22/09/10 10:44:49 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 961283 ms exceeds timeout 120000 ms
22/09/10 10:44:49 WARN SparkContext: Killing executors is not supported by current scheduler.


no outliers found

In [19]:
data.write.mode('overwrite').parquet('../data/curated/outlier_data.parquet')